In [16]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker


import pynwb
from pynwb import register_class
from pynwb.form.utils import docval, getargs, popargs, call_docval_func

import nspike_helpers as ns 
import query_helpers as qu

mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Spec for a Behavioral World group

In [17]:
behavioral_world_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralWorld',
                                               doc='Collection of geometries (1D, 2D, or 3D) describing the behavioral environment',
                                               neurodata_type_inc='NWBContainer',
                                               neurodata_type_def='FL_BehavioralWorld')

behavioral_fsm_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralFSM',
                                               doc='Finite state machine describing non-overlapping behavioral states',
                                               neurodata_type_inc='NWBContainer',
                                             neurodata_type_def='FL_BehavioralFSM')

behavioral_events_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralEvents',
                                               doc='DynamicTable of describing possibly-overlapping behavioral/experimental events',
                                               neurodata_type_inc='DynamicTable',
                                                neurodata_type_def='FL_BehavioralEvents')

### Spec for a Behavioral Task group
Contains: 
* Beahvioral world - geometry describing the spatial and operant world of this task (e.g. polygons, lines, points representing 1D and 2D track geometry, 3D room, notable objects or cues). These world geometries can then be used as reference frames when describing animal position.
* Behavioral FSM - discrete, non-overlapping states of animal behavior (e.g. open-field no well, open-field at well, arm 4 no well, arm 4 at well, ...)
* Experimental events - time interval instances of experimental/behavioral events (e.g. reward delivered, cue light on), which could be overlapping



In [18]:
task_groups = [pynwb.spec.NWBGroupSpec(name='world',
                                       doc='Collection of geometries (1D, 2D, or 3D) describing the behavioral environment',
                                       neurodata_type_inc='FL_BehavioralWorld',
                                       quantity='?'),
               pynwb.spec.NWBGroupSpec(name='fsm',
                                       doc='Finite state machine describing non-overlapping behavioral states',
                                       neurodata_type_inc='FL_BehavioralFSM',
                                       quantity='?'),
               pynwb.spec.NWBGroupSpec(name='events',
                                       doc='DynamicTable of describing possibly-overlapping behavioral/experimental events',
                                       neurodata_type_inc='FL_BehavioralEvents',
                                       quantity='?')]

behavioral_task_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralTask',
                                               doc='Container of objects for describing a behavioral task',
                                               groups=task_groups,
                                               neurodata_type_inc='NWBContainer',
                                               neurodata_type_def='FL_BehavioralTask')

### Spec for an Epochs DynamicTable

In [19]:
epoch_datasets = [pynwb.spec.NWBDatasetSpec(name='epoch_name',
                                            doc='Name for this epoch',
                                            dtype='text',
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='task',
                                            doc='Task used in this epoch',
                                            dtype=pynwb.spec.NWBRefSpec('FL_BehavioralTask', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='position', 
                                            doc='Animal spatial position during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('SpatialSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='speed', 
                                            doc='Animal speed during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('TimeSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='head_dir', 
                                            doc='Animal head direction during an epoch',
                                            dtype=pynwb.spec.NWBRefSpec('SpatialSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData')]

behavioral_epochs_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralEpochs',
                                                 doc='DynamicTable for describing behavioral epochs',
                                                 neurodata_type_inc='DynamicTable', 
                                                 neurodata_type_def='FL_BehavioralEpochs',
                                                 datasets=epoch_datasets,
                                                 quantity="?")




### Build a namespace and export to YAML

In [20]:
ns_path = "franklab.namespace.yaml"
ext_path = "franklab.extensions.yaml"
ns_builder = pynwb.spec.NWBNamespaceBuilder('Extension for use in the Frank Lab', "franklab")
ns_builder.add_spec(ext_path, behavioral_world_spec)
ns_builder.add_spec(ext_path, behavioral_fsm_spec)
ns_builder.add_spec(ext_path, behavioral_events_spec)
ns_builder.add_spec(ext_path, behavioral_task_spec)
ns_builder.add_spec(ext_path, behavioral_epochs_spec)
ns_builder.export(ns_path)

### Load our namespace, auto-generate a Python class

In [8]:
pynwb.load_namespaces(ns_path)

('franklab',)

### Create a Python class for this DynamicTable
The class autogeneration function does not create a correct class.

In [9]:
@register_class('FL_BehavioralEpochs', 'franklab')
class FL_BehavioralEpochs(pynwb.core.DynamicTable):
    """
    Data for behavioral epochs.
    """

    # Setting index to True throws an error if I try to store non-iterable types, such as SpatialSeries
    __columns__ = (
        {'name': 'epoch_name', 'description': 'Name for this epoch', 'index': False},
        {'name': 'position', 'description': 'Animal spatial position during an epoch', 'index': False},
        {'name': 'speed', 'description': 'Animal speed during an epoch', 'index': False},
        {'name': 'head_dir', 'description': 'Animal head direction during an epoch', 'index': False}
    )

    @docval({'name': 'name', 'type': str, 'doc': 'Name of this FL_BehavioralEpochs table', 'default': 'FL_BehavioralEpochs'},
            {'name': 'id', 'type': ('array_data', pynwb.core.ElementIdentifiers),
             'doc': 'the identifiers for the epochs stored in this interface', 'default': None},
            {'name': 'columns', 'type': (tuple, list), 'doc': 'the columns in this table', 'default': None},
            {'name': 'colnames', 'type': 'array_data', 'doc': 'the names of the columns in this table',
             'default': None},
            {'name': 'description', 'type': str, 'doc': 'a description of what is in this FL_BehavioralEpochs table', 'default': None})
    def __init__(self, **kwargs):
        if kwargs.get('description', None) is None:
            kwargs['description'] = ""
        call_docval_func(super(FL_BehavioralEpochs, self).__init__, kwargs)

    @docval({'name': 'epoch_name', 'type': str, 'doc': 'Name for this epoch', 'default': None},
            {'name': 'position', 'type': pynwb.behavior.SpatialSeries, 'doc': 'Animal spatial position during an epoch', 'default': None},
            {'name': 'speed', 'type': pynwb.base.TimeSeries, 'doc': 'Animal speed during an epoch', 'default': None},
            {'name': 'head_dir', 'type': pynwb.behavior.SpatialSeries, 'doc': 'Animal head direction during an epoch', 'default': None},
            {'name': 'id', 'type': int, 'default': None, 'help': 'the id for each unit'},
            allow_extra=True)
    def add_epoch(self, **kwargs):
        """
        Add an epoch to this table
        """
        super(FL_BehavioralEpochs, self).add_row(**kwargs)


### Extract behavioral data

In [10]:
# Session-specific params
data_dir = os.path.expanduser('~/Data/FrankData/kkay/Bon')
anim = 'Bon' 
prefix = anim.lower()
day = 3 # below we'll code date as 2006-Jan-'Day'

# Calculate the POSIX timestamp when Nspike clock = 0 (seconds)
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))
NSpike_posixtime_offset = dataset_zero_time.timestamp()

# NOTE that day_inds is 0 based
time_list = {}
nwb_epoch = {}
pos_files = ns.get_files_by_day(data_dir, prefix, 'pos')
task_files = ns.get_files_by_day(data_dir, prefix, 'task')

mat = ns.loadmat_ff(task_files[day], 'task')
task_struct = mat[day]

mat = ns.loadmat_ff(pos_files[day], 'pos')
pos_struct = mat[day]

# create position, direction and speed
position_list = []
direction_list = []
speed_list = []
time_list = []

# Assume field order: (time,x,y,dir,vel)
(time_idx, x_idx, y_idx, dir_idx, vel_idx) = range(5)

for epoch_num, pos_epoch in pos_struct.items():

    # convert times to POSIX time
    timestamps = pos_epoch['data'][:,time_idx] + NSpike_posixtime_offset

    # TODO: create a shared TimeSeries for timestamps, across all behavioral timeseries
    # ?? timestamps_obj = pynwb.TimeSeries(timestamps=timestamps...)

    # collect times of epoch start and end
    time_list.append([timestamps[0], timestamps[-1]])

    m_per_pixel = pos_epoch['cmperpixel'][0,0]/100 # NWB wants meters per pixel

    # we can also create new SpatialSeries for the position, direction and velocity information
    #NOTE: Each new spatial series has to have a unique name.
    position_list.append(pynwb.behavior.SpatialSeries(name='Position d%d e%d' % (day, epoch_num), 
                              timestamps = timestamps,
                              data=pos_epoch['data'][:, (x_idx, y_idx)] * m_per_pixel,
                              reference_frame='corner of video frame',
                              #conversion=m_per_pixel,
                              #unit='m'
                              )) # *after* conversion

    direction_list.append(pynwb.behavior.SpatialSeries(name='Head Direction d%d e%d'% (day, epoch_num), 
                              timestamps=timestamps,
                              data=pos_epoch['data'][:, dir_idx],
                              reference_frame='0=facing top of video frame (?), positive clockwise (?)',
                              #unit='radians'
                              ))

    speed_list.append(pynwb.base.TimeSeries(name='Speed d%d e%d' % (day, epoch_num),
                             timestamps=timestamps,
                             data=pos_epoch['data'][:, vel_idx] * m_per_pixel,
                             unit='m/s',
                             #conversion=m_per_pixel,
                             description='smoothed movement speed estimate'))
time_list = np.asarray(time_list)

### Implement an Epochs table using our class

In [106]:
epochs = FL_BehavioralEpochs()
for epoch_idx in range(len(pos_struct)):
    epochs.add_epoch(epoch_name='Epoch ' + str(epoch_idx), 
                     position=position_list[epoch_idx], 
                     speed=speed_list[epoch_idx], 
                     head_dir=direction_list[epoch_idx])
    

### Inspect our Epochs table

In [128]:
epochs[0, 'position']


Position d3 e1 <class 'pynwb.behavior.SpatialSeries'>
Fields:
  comments: no comments
  conversion: 1.0
  data: [[1.21095 0.87075]
 [1.215   0.86265]
 [1.215   0.86265]
 ...
 [0.8667  0.83835]
 [0.8667  0.83835]
 [0.8667  0.83835]]
  description: no description
  interval: 1
  num_samples: 33753
  reference_frame: corner of video frame
  resolution: 0.0
  timestamps: [1.13632087e+09 1.13632087e+09 1.13632087e+09 ... 1.13632199e+09
 1.13632199e+09 1.13632199e+09]
  timestamps_unit: Seconds
  unit: meters